In [ ]:
from google.auth import default
from google.colab import auth
from google.colab import files
import gspread
import pandas as pd

auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# The URLs for the spreadsheets that catch all the form data for art, theme camps, and events

# artURL = 'https://docs.google.com/spreadsheets/d/1nWnP165f-8EETuoejS5DiRA4g46WtAUlUgH6ISY8T4E/edit#gid=2113055750'
# eventURL = 'https://docs.google.com/spreadsheets/d/1Tb7Gxxl8iXwh8d7dVlWJJA9moGGu4VoakFafesDCFVU/edit#gid=1769843369'
# campURL = 'https://docs.google.com/spreadsheets/d/11mBjvuw1JLgF6LeGYzaDwemQrYC6qsyhbPLJpk9aiAM/edit#gid=91151219'
artURL = 'https://docs.google.com/spreadsheets/d/1QObtQgGYhetnxALQq4kIwrgBCHkxTPwzQw5OKHM4K68/edit?usp=sharing'
eventURL = 'https://docs.google.com/spreadsheets/d/1fFnxr5e3VtNUvGY7011j_heS3xffzw9dltIJcrPORmM/edit?usp=sharing'
campURL = 'https://docs.google.com/spreadsheets/d/1ZnY8mX8xyanx8sLYMZd-Kx54Wsv3hS0n64p-Rijp5do/edit#gid=486141219'

# Process Art

In [ ]:
# read data and put it in a dataframe
gs = gc.open_by_url(artURL)
worksheet = gs.get_worksheet(0)

df = pd.DataFrame(worksheet.get_all_records())
df.rename(columns={'Name of Art ': 'title',
                   'Artist ': 'artist',
                   'Location' : 'camp',
                   'Public Art Description': 'description'}, inplace=True)
df = df[['title', 'artist', 'camp', 'description']]
df.sort_values(by='title', inplace=True)

output = ''

for index, row in df.iterrows():
  title =  '\subsection*{' + '{}'.format(row[0].strip()) + '}\n'
  artist = '{\color{violet} \\faPalette} ' + '{}'.format(row[1].strip()) + '\n'
  camp = row[2].strip()
  campinfo = ''
  if camp != '':
    campinfo = '{\color{teal} \\faMapMarked} ' + '{}'.format(camp) + '\n' # make this a fancy location marker
  description =  '{}'.format(row[3].strip()) + '\n'
  output += '\\vbox{\n' + title + artist + campinfo + description + '}\n\n'

  # break
output = output.replace('&', '\\&')
output = output.replace('\n', '\n\n')
# print(output)

with open('art_raw.tex', 'w') as f:
  f.write(output)

files.download('art_raw.tex')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Process Camps -- in progress

In [ ]:
def makeCampTheme(theme):
  icons = ''
  if 'Adult' in theme:
    icons += '\\faUserAstronaut '
  if 'Kids' in theme:
    icons += '\\faChild '
  if 'Art' in theme:
    icons += '\\faPalette '
  if 'Bar' in theme:
    icons += '\\faIcon{glass-martini-alt} '
  if 'Drinks-non-alcoholic' in theme:
    icons += '\\faCoffee '
  if 'Food' in theme:
    icons += '\\faPizzaSlice '
  if 'Chill' in theme:
    icons += '\\faUmbrellaBeach '
  if 'Games' in theme:
    icons += '\\faDice '
  if 'Learning' in theme:
    icons += '\\faGraduationCap '
  if 'Music-silent' in theme:
    icons += '\\faHeadphones '
  elif 'Music' in theme:
    icons += '\\faMusic '
  if 'Performance' in theme:
    icons += '\\faTheaterMasks '
  return icons
  # \\faChild \\faBaby \\faBasketballBall \\faCameraRetro \\faCampground \\faCoffee \\faCouch \\faDice \\faHiking \\faLeaf \\faUmbrellaBeach \\faWalking
  # \\faIcon[regular]{lightbulb} \\faMoon \\faMortarPestle \\faMugHot \\faPeace \\faRainbow \\faRunning \\faSeedling  \\faStar \\faSun \faUserAstronaut \\faUser


def compileCampOutput(camp, theme, description, location='', notes=''):
  if camp in ["We're Occult", 'World Spirits']:
    output = '\\vbox{\n' + '\subsection*{\\begin{tblr}{Q[0.6\columnwidth]X[halign=r, valign=t]}' + '{}'.format(camp.replace('&', '\\&')) + '& {\color{purple} ' + \
             theme + '}\end{tblr}\\vspace{-1em}}\n' + location + description.replace('&', '\\&').replace('#', '\\#') + notes + '\n}\n\n'
  else:
    output = '\\vbox{\n' + '\subsection*{\\begin{tblr}{Q[0.7\columnwidth]X[halign=r, valign=t]}' + '{}'.format(camp.replace('&', '\\&')) + '& {\color{purple} ' + \
             theme + '}\end{tblr}\\vspace{-1em}}\n' + location + description.replace('&', '\\&').replace('#', '\\#') + notes + '\n}\n\n'
  output = output.replace('\n', '\n\n')
  return output


# read data and put it in a dataframe
gs = gc.open_by_url(campURL)
worksheet = gs.get_worksheet(0)

df = pd.DataFrame(worksheet.get_all_records())
df.rename(columns={'Theme Camp Name': 'camp',
                   'In what "genres" would you classify your camp? Check all that apply.': 'theme',
                  #  'Notes': 'notes',
                   'Public Camp Description': 'description'}, inplace=True)
df = df[['camp', 'theme', 'description']] # update to reflect inputs -- note that we could add neighborhoods here if we have those
df.sort_values(by='camp', inplace=True)

output = ''

for index, row in df.iterrows():
  camp = row[0].strip()
  theme = makeCampTheme(row[1].strip())
  description =  '{}'.format(row[2].strip()) + '\n'
  # location = '{\color{teal} \\faMapMarked}~~' + row[3].strip() + '\n'
  # notes =  '{}'.format(row[4].strip())
  # if row[4].strip() == 'skip':
  #   continue
  # elif notes != '':
  #   notes = '\n\\faInfoCircle~~' + notes
  output += compileCampOutput(camp, theme, description)

with open('camps_raw.tex', 'w') as f:
  f.write(output)

files.download('camps_raw.tex')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Process Events

In [ ]:
# read data and put it in a dataframe
gs = gc.open_by_url(eventURL)
worksheet = gs.get_worksheet(0)

df = pd.DataFrame(worksheet.get_all_records())
df.rename(columns={'What shall we call your Event?': 'title',
                   'Who is hosting? Theme Camp or Your name': 'host',
                   'Which day would you prefer? ': 'day',
                   'What time slot would you prefer? ': 'time_pretty',
                   'Description of your play-learn-workshop-event for the Pocket Guide!': 'description',
                   'Select a theme of your event for Pocket Guide!': 'theme',
                   'If Materials Are Used, Will You Provide Them?' : 'bring',
                   'Not a Theme Camp? Please give us a location (art installation, lakeside, effigy, temple.. etc)': 'location'}, inplace=True)
df = df[['title', 'host', 'location', 'day', 'time_pretty', 'time_tech', 'theme', 'description', 'bring']]
df = df.infer_objects()
df.sort_values(by='time_tech', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 50 to 27
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        53 non-null     object
 1   host         53 non-null     object
 2   location     53 non-null     object
 3   day          53 non-null     object
 4   time_pretty  53 non-null     object
 5   time_tech    53 non-null     int64 
 6   theme        53 non-null     object
 7   description  53 non-null     object
 8   bring        53 non-null     object
dtypes: int64(1), object(8)
memory usage: 4.1+ KB


In [ ]:
def makeEventTheme(theme):
  if theme == 'Performance':
    return '{\color{purple} \\faTheaterMasks}'
  elif theme == 'Event':
    return '{\color{purple} \\faIcon[regular]{calendar-alt}}'
  elif theme == 'Party':
    return '{\color{purple} \\faIcon{glass-martini-alt}}'
  elif theme == 'Workshop':
    return '{\color{purple} \\faGraduationCap}'
  elif theme == 'Game':
    return '{\color{purple} \\faChess}'
  elif theme == 'Music':
    return '{\color{purple} \\faMusic}'
  elif theme == 'Food':
    return '{\color{purple} \\faPizzaSlice}'
  elif theme == 'Non-alcoholic-Drinks':
    return '{\color{purple} \\faCoffee}'
  elif theme == 'Tour':
    return '{\color{purple} \\faIcon{bus-alt}}'
  elif theme == 'Fire':
    return '{\color{purple} \\faIcon{fire-alt}}'
  elif theme == 'Chill':
    return '{\color{purple} \\faUmbrellaBeach}'
  else:
    return ''

def compileEventOutput(title, host, location, time, description, bring):
  output = '\\vbox{\n' + title + "\\begin{description}[leftmargin=2em,noitemsep,style=nextline]\n" + host + location + time +\
            bring + "\end{description}\n" + description.replace('&', '\\&') + '}\n\n'
  # output = output.replace('&', '\\&')
  output = output.replace('\n', '\n\n')
  return output


ongoing = '' # title for ongoing saved separately
thursday = '\section[Thursday]{Thursday Events}\n\n'
friday = '\section[Friday]{Friday Events}\n\n'
saturday = '\section[Saturday]{Saturday Events}\n\n'
sunday = '\section[Sunday]{Sunday Events}\n\n'

In [ ]:
for index, row in df.iterrows():
  title =  '\subsection*{\\begin{tblr}{Q[0.8\columnwidth]X[halign=r, valign=t]}' + '{} & {}'.format(row[0].strip().replace('&', '\\&'), makeEventTheme(row[6].strip())) + '\end{tblr}}\n'
  host = '\item[{\color{violet} \\faUserFriends}] ' + '{}'.format(row[1].strip()) + '\n'
  locationField = row[2].strip()
  location = ''
  if locationField != '':
    location = '\item[{\color{teal} \\faMapMarked}] ' + '{}'.format(locationField) + '\n'
  # theme = makeEventTheme(row[6])
  day = row[3].strip().split()[0].split(',')[0]
  time = '\item[{\color{cyan} \\faClock[regular]}] ' + '{}'.format(row[4].strip()) + '\n'
  timeTech = '{:04d}'.format(row[5])
  description =  '{}'.format(row[7].strip()) + '\n'
  bring = ''
  if row[8].strip() != '':
    bring = '\item[{\color{red} \\faSuitcase}] ' + '{}'.format(row[8].strip()) + '\n'
  output = compileEventOutput(title, host, location, time, description, bring)
  if day == 'Everyday':
    ongoing += output
  elif day == 'Thursday':
    thursday += output
  elif day == 'Friday':
    friday += output
  elif day == 'Saturday':
    saturday += output
  elif day == 'Sunday':
    sunday += output


with open('events-raw.tex', 'w') as f:
  f.write(ongoing + thursday + friday + saturday + sunday)

files.download('events-raw.tex')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>